In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
# Il faut ajouter le fichier "adult.csv" qui n'est pas inclut dans notre remise
def load_original():
    return pd.read_csv("adult.csv")

# Voir `preprocess(df)` plus bas
def load_modified():
    return pd.read_csv("adult-modified.csv")

#df = load_original()
df = load_modified()

In [3]:
df.describe(include='all')

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
count,48842.000000,48842,4.884200e+04,48842,48842.000000,48842,48842,48842,48842,48842,48842.000000,48842.000000,48842.000000,48842,48842
unique,NaN,9,NaN,16,NaN,7,15,6,5,2,NaN,NaN,NaN,42,2
top,NaN,Private,NaN,HS-grad,NaN,Married-civ-spouse,Prof-specialty,Husband,White,Male,NaN,NaN,NaN,United-States,<=50K
freq,NaN,33906,NaN,15784,NaN,22379,6172,19716,41762,32650,NaN,NaN,NaN,43832,37155
mean,39.165268,NaN,1.939211e+05,NaN,10.078089,NaN,NaN,NaN,NaN,NaN,5711.725564,295.394230,43.844642,NaN,NaN
std,14.060112,NaN,1.125794e+05,NaN,2.570973,NaN,NaN,NaN,NaN,NaN,7227.660657,361.260868,12.818377,NaN,NaN
min,15.000000,NaN,7.452000e+04,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,4999.500000,217.500000,5.000000,NaN,NaN
25%,25.000000,NaN,7.452000e+04,NaN,9.000000,NaN,NaN,NaN,NaN,NaN,4999.500000,217.500000,45.000000,NaN,NaN
50%,35.000000,NaN,2.235600e+05,NaN,10.000000,NaN,NaN,NaN,NaN,NaN,4999.500000,217.500000,45.000000,NaN,NaN
75%,45.000000,NaN,2.235600e+05,NaN,12.000000,NaN,NaN,NaN,NaN,NaN,4999.500000,217.500000,45.000000,NaN,NaN


### A-priori
A-priori sera utilisé ici pour trouver les patrons les plus fréquents.

In [4]:
def preprocess(df):
    # Je veux ici réduire le nombre de valeur possible
    # Parce que des patrons différents pour 50 ans et 51 ans ne
    # devrait pas vraiment être différents
    for age in df['age'].unique():
        df.loc[df['age'] == age, 'age'] = ((age // 10) * 10 + 5)
    
    gain_step = df['capital-gain'].max() // 10
    loss_step = df['capital-loss'].max() // 10
    
    for gain in df['capital-gain'].unique():
        df.loc[df['capital-gain'] == gain, 'capital-gain'] = ((gain // gain_step) * gain_step + (gain_step / 2))
    
    for loss in df['capital-loss'].unique():
        df.loc[df['capital-loss'] == loss, 'capital-loss'] = ((loss // loss_step) * loss_step + (loss_step / 2))
    
    fnlwgt_step = df['fnlwgt'].max() // 10
    
    for w in df['fnlwgt'].unique():
        df.loc[df['fnlwgt'] == w, 'fnlwgt'] = ((w // fnlwgt_step) * fnlwgt_step + (fnlwgt_step / 2))
    
    for h in df['hours-per-week'].unique():
        df.loc[df['hours-per-week'] == h, 'hours-per-week'] = ((h // 10) * 10 + 5)
    
    df.to_csv('adult-modified.csv', sep=',', encoding='utf-8', index=False)

In [5]:
#preprocess(df)

`patterns` va ici contenir les éléments les plus fréquents pour chaque colonne:

In [6]:
def get_most_frequent(data, count):
    patterns = dict()
    for c in data: # iterate over columns
        # get the most frequent values
        # `value_counts` sort in descending order
        patterns[c] = data[c].value_counts()
        for r in patterns[c]: # iterate over rows
            if len(patterns[c]) >= count:
                patterns[c] = patterns[c].iloc[:count]
    #print(patterns)
    return patterns

In [7]:
def apriori_impl(data, num_key, max_id_key, pattern_min, done_index):
    if num_key <= 0:
        # Retourne un type différent de `result` !!!
        return data.describe(include='all') #return data # A leaf
    patterns = get_most_frequent(data, max_id_key)
    result = dict()
    for key in patterns.keys():
        if (key not in done_index):
            done_index |= set([key])
            for idx in range(len(patterns[key].index)):
                value = patterns[key].index[idx]
                #print(key + ":" + str(value))
                data_c = data[data[key] == patterns[key].index[idx]]
                #print(data_c.shape)
                if (data_c.shape[0] >= pattern_min):
                    result[(key, value)] = apriori_impl(data_c,num_key - 1,max_id_key,pattern_min,done_index)
    return result

def apriori(data, num_key, max_id_key, pattern_min):
    return apriori_impl(data, num_key, max_id_key, pattern_min, done_index=set())

In [8]:
#print(df.iloc[df.index.difference(['age','capital-gain'])].describe())

num_key = 2 # Nombre de clef dans un pattron
pattern_min = 1000 # Nombre minimal pour être un patron
max_id_key = 2 # Quantité de chaque clef à considérer

# pattern = apriori(df, num_key, max_id_key, pattern_min)

# print(pattern) trop gros pour imprimer

In [9]:
pd.set_option('float_format', '{:.1f}'.format)

def print_pattern(pattern):
    ks = pattern.keys()
    for k in range(len(ks)):
        key = list(pattern.keys())[k]
        if type(pattern[key]) != dict:
            display(pattern[key])
        else:
            print_pattern(pattern[key])

#print_pattern(pattern)

### Aspects éthiques

In [10]:
male = df[df['gender'] == 'Male']
female = df[df['gender'] == 'Female']

rich_male = male[male['income'] == '>50K'].shape[0]
rich_female = female[female['income'] == '>50K'].shape[0]
poor_male = male[male['income'] == '<=50K'].shape[0]
poor_female = female[female['income'] == '<=50K'].shape[0]

print(df.groupby('income').mean())
print("Ratio >50K Homme/Femme == %.4f" % (rich_male / rich_female))
print("Ratio <=50K Homme/Femme == %.4f" % (poor_male / poor_female))

        age   fnlwgt  educational-num  capital-gain  capital-loss  \
income                                                              
<=50K  37.4 194217.3              9.6        5010.3         264.9   
>50K   44.8 192979.2             11.6        7941.8         392.4   

        hours-per-week  
income                  
<=50K             42.2  
>50K              48.9  
Ratio >50K Homme/Femme == 5.6066
Ratio <=50K Homme/Femme == 1.5761


#### Discussion
Ces chiffres montrent qu'il y a plus de 5 fois plus d'homme que de femme avec un salaire en haut de 50 000 tandis qu'il y a moins de 2 fois plus d'homme avec un salaire en dessous de 50 000.

Il y a deux possibilités: peut-être que ces données sont erronées (ou incomplètes) ou qu'il y a vraiment une différence entre homme et femme. Si les données ne sont pas erronées ni incomplètes alors ce jeu de données pourrait dans certains contextes être utilisé pour faire des prédictions. Il y a quand même une nuance à faire: si on utilise ce jeu de données pour prendre des décisions alors il faut se demander si cette différence est pertinente. Sans compter que le jeu de données pourrait être le reflet de plusieurs autres phénomènes sociaux plus ou moins pertinents. C'est à dire que toutes décisions prisent automatiquement à l'aide d'un algorithme pourraient être basées en partie sur ces problèmes éthiques plutôt que sur des éléments rationnels et/ou sur des valeurs communément acceptées.

En d'autres termes, des données représentatives peuvent dans certains contextes être utilisées pour faire des prédictions. Le problème se corse lorsqu'on veut aussi prendre une décision. Par exemple, si on veut prendre une décision sur les salaires dans une entreprise il faut se demander si ces caractérisques doivent réellement être considérées. En générale on voudrait probablement considérer l'éducation pour déterminer un salaire mais pas le sexe. Il faut donc prendre soin de sélectionner des caractérisques pertinentes. Dans le cas contraire et comme nous avons vue dans le cours, il peut exister un indice de corrélation non null sur des données qui ne sont pas du tout en lien avec la décision qu'on s'aprète à prendre.

Si ces données reste pertinentes dans certains cas c'est parce qu'il n'est pas toujours facile de faire la distinction entre une relation de causation et une corrélation due au hasard. Il est à la fois possible de trouver des corrélations duent au hasard, des corrélations duent à des problèmes éthiques et des corrélations duent à une relation de causation pertinente. Il y a quand même une relation de causation si la corrélations est due à des problèmes éthiques mais la corrélation n'est pas nécessairement pertinente pour la question abordée.